In [85]:
import pandas as pd

pbp = pd.read_csv('data/pbp/2018.csv', sep=",", encoding='iso-8859-1')
print(pbp.id.nunique())
pbp.info()

153544
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153544 entries, 0 to 153543
Data columns (total 19 columns):
id                    153544 non-null int64
offense               153544 non-null object
offense_conference    143889 non-null object
defense               153544 non-null object
defense_conference    142945 non-null object
home                  153544 non-null object
away                  153544 non-null object
offense_score         153544 non-null int64
defense_score         153544 non-null int64
drive_id              153544 non-null int64
period                153544 non-null int64
clock.minutes         153544 non-null int64
clock.seconds         153544 non-null int64
yard_line             153544 non-null int64
down                  153544 non-null int64
distance              153544 non-null int64
yards_gained          153544 non-null int64
play_type             153544 non-null object
play_text             153544 non-null object
dtypes: int64(11), object(8)
memory us

In [86]:
# Cleaning
ignore_types = ["Punt Return Touchdown","Defensive 2pt Conversion","Fumble Return Touchdown","Sack","Pass Interception Return","Interception Return Touchdown","Interception","Fumble Recovery (Own)","Blocked Field Goal","Blocked Punt","Fumble Recovery (Opponent)","Missed Field Goal Return","Blocked Punt Touchdown","Missed Field Goal Return Touchdown","Extra Point Missed","Extra Point Good","Timeout","End of Half","End of Game","Uncategorized","Penalty","Kickoff","Kickoff Return (Offense)","Kickoff Return Touchdown","Punt", "Field Goal Good","Field Goal Missed","Safety"]

bad_data = pbp[
    (pbp.down == 0)
    | (pbp.play_type.isin(ignore_types))
    | ((abs(pbp.offense_score - pbp.defense_score) >= 38) & (pbp.period == 2))
    | ((abs(pbp.offense_score - pbp.defense_score) >= 28) & (pbp.period == 3))
    | ((abs(pbp.offense_score - pbp.defense_score) >= 22) & (pbp.period == 4))
]

print(pbp.id.nunique())
print(bad_data.id.nunique())

pbp.drop(bad_data.index, inplace=True)
print(pbp.id.nunique())

153544
55612
97932


In [87]:
import numpy as np

def is_successful(down, distance, yards_gained):
    if ((down == 1) & (yards_gained >= (0.5 * distance))):
        return True
    elif ((down == 2)) & (yards_gained >= (0.7 * distance)):
        return True
    elif ((down == 3) & (yards_gained >= distance)):
        return True
    else:
        return False
    
pbp['play_successful'] = np.vectorize(is_successful)(pbp.down, pbp.distance, pbp.yards_gained)

pbp.play_type.value_counts()

Rush                 50038
Pass Reception       25353
Pass Incompletion    17600
Rushing Touchdown     2494
Passing Touchdown     2447
Name: play_type, dtype: int64

In [88]:
pass_types = ["Pass Reception","Pass Incompletion","Passing Touchdown"]
rush_types = ["Rush","Rushing Touchdown"]

league_pass_plays = pbp[
    pbp.play_type.isin(pass_types)
]
league_rush_plays = pbp[
    pbp.play_type.isin(rush_types)
]
print(league_pass_plays.id.nunique())
print(league_rush_plays.id.nunique())

45400
52532


In [89]:
print(league_pass_plays.play_successful.value_counts(normalize=True))
print(league_pass_plays.play_successful.value_counts())

False    0.97304
True     0.02696
Name: play_successful, dtype: float64
False    44176
True      1224
Name: play_successful, dtype: int64


In [90]:
print(league_rush_plays.play_successful.value_counts(normalize=True))
print(league_rush_plays.play_successful.value_counts())

False    0.948964
True     0.051036
Name: play_successful, dtype: float64
False    49851
True      2681
Name: play_successful, dtype: int64


In [93]:
pbp[
   ((pbp.down == 1) & (pbp.yards_gained == (0.5 * pbp.distance)))
    | ((pbp.down == 2)) & (pbp.yards_gained == (0.7 * pbp.distance))
    | ((pbp.down == 3) & (pbp.yards_gained == pbp.distance))
].drive_id.info()

AttributeError: 'Series' object has no attribute 'info'